In [5]:
from os.path import expanduser
import os

import pandas as pd
from pyarrow import parquet as pq
from datatable import dt, f
from alpha_vantage.timeseries import TimeSeries
from zlogin import fetch_access_token, fetch_kiteconnect_instance, fetch_kiteticker_instance
from kiteconnect import KiteConnect

# Get Zerodha, AlphaVantage Access Points

In [6]:
with open(expanduser('~/.tradesecrets'), 'r') as file:
    env_vars = [line.strip().split('=') for line in file.readlines()]

for pair in env_vars:
    key, value = pair
    os.environ[key] = value

ALPHAVANTAGE_API_KEY = os.environ['ALPHA_KEY']

ts = TimeSeries(key=ALPHAVANTAGE_API_KEY, output_format='pandas')
kite = fetch_kiteconnect_instance()

Access Token Returned!


# Zerodha Instruments

In [20]:
all_instruments = kite.instruments()

# Explore instruments from Zerodha

In [21]:
instruments = dt.Frame(list(map(
        lambda x: {k: v if v != '' else None for k, v in x.items()}, all_instruments))
    )
instruments

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,546629638,2135272,EURINR22DEC74.75CE,EURINR,0,2022-12-28,74.75,0.0025,1,CE,BCD-OPT,BCD
1,546627846,2135265,EURINR22DEC74.75PE,EURINR,0,2022-12-28,74.75,0.0025,1,PE,BCD-OPT,BCD
2,546356486,2134205,EURINR22DEC75.25CE,EURINR,0,2022-12-28,75.25,0.0025,1,CE,BCD-OPT,BCD
3,546355462,2134201,EURINR22DEC75.25PE,EURINR,0,2022-12-28,75.25,0.0025,1,PE,BCD-OPT,BCD
4,546383622,2134311,EURINR22DEC75.5CE,EURINR,0,2022-12-28,75.5,0.0025,1,CE,BCD-OPT,BCD
5,546382342,2134306,EURINR22DEC75.5PE,EURINR,0,2022-12-28,75.5,0.0025,1,PE,BCD-OPT,BCD
6,546196486,2133580,EURINR22DEC75.75CE,EURINR,0,2022-12-28,75.75,0.0025,1,CE,BCD-OPT,BCD
7,546192902,2133566,EURINR22DEC75.75PE,EURINR,0,2022-12-28,75.75,0.0025,1,PE,BCD-OPT,BCD
8,546387718,2134327,EURINR22DEC75CE,EURINR,0,2022-12-28,75,0.0025,1,CE,BCD-OPT,BCD


In [254]:
instruments.shape, list(zip(instruments.names, map(str,instruments.types)))


((101006, 12),
 [('instrument_token', 'Type.int32'),
  ('exchange_token', 'Type.str32'),
  ('tradingsymbol', 'Type.str32'),
  ('name', 'Type.str32'),
  ('last_price', 'Type.float64'),
  ('expiry', 'Type.date32'),
  ('strike', 'Type.float64'),
  ('tick_size', 'Type.float64'),
  ('lot_size', 'Type.int32'),
  ('instrument_type', 'Type.str32'),
  ('segment', 'Type.str32'),
  ('exchange', 'Type.str32')])

In [256]:
instruments[:, dt.count(), dt.by(f['exchange'], f.segment, f.instrument_type)]#.to_pandas().to_clipboard()


,exchange,segment,instrument_type,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,BCD,BCD,EQ,451
1,BCD,BCD-FUT,FUT,60
2,BCD,BCD-OPT,CE,1568
3,BCD,BCD-OPT,PE,1386
4,BSE,BSE,EQ,11108
5,BSE,INDICES,EQ,55
6,CDS,CDS-FUT,FUT,152
7,CDS,CDS-OPT,CE,6177
8,CDS,CDS-OPT,PE,6177


In [257]:
instruments[f.segment == 'NFO-FUT',:][:,dt.count(),dt.by(f.name)][:,:,dt.sort(-f.count)]

,name,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,MIDCPNIFTY,11
1,AARTIIND,3
2,ABB,3
3,ABBOTINDIA,3
4,ABCAPITAL,3
5,ABFRL,3
6,ACC,3
7,ADANIENT,3
8,ADANIPORTS,3


In [258]:
instruments[(f.segment == 'NFO-OPT') & (f.name=='NIFTY'),:][:,dt.count(),dt.by('expiry')]

,expiry,count
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,2022-09-29,232
1,2022-10-06,242
2,2022-10-13,232
3,2022-10-20,220
4,2022-10-27,242
5,2022-11-03,218
6,2022-11-10,208
7,2022-11-17,204
8,2022-11-24,220


In [259]:
instruments[(f.exchange == 'NSE') & (f.segment=='INDICES'),:][:,['name','tradingsymbol']]

,name,tradingsymbol
,▪▪▪▪,▪▪▪▪
0,HANGSENG BEES-NAV,HANGSENG BEES-NAV
1,INDIA VIX,INDIA VIX
2,NIFTY 100,NIFTY 100
3,NIFTY 200,NIFTY 200
4,NIFTY 50,NIFTY 50
5,NIFTY 500,NIFTY 500
6,NIFTY ALPHA 50,NIFTY ALPHA 50
7,NIFTY ALPHALOWVOL,NIFTY ALPHALOWVOL
8,NIFTY AUTO,NIFTY AUTO


In [260]:
futures_underlying = instruments[f.segment == 'NFO-FUT',:]['name'][0,:, dt.by('name')].to_list()[0]
current_underlying = instruments[(f.exchange == 'NSE') #& (f.name != None)
                                , :][:, f['tradingsymbol']][0, :, dt.by('tradingsymbol')].to_list()[0]
subset = set(futures_underlying) - set(current_underlying)


In [261]:
instruments[(f.segment == 'NFO-FUT') &
            ((f.name == 'FINNIFTY') | (f.tradingsymbol == 'NIFTY')), :]


,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,9922818,38761,FINNIFTY22NOVFUT,FINNIFTY,0,2022-11-29,0,0.05,40,FUT,NFO-FUT,NFO
1,14144770,55253,FINNIFTY22OCTFUT,FINNIFTY,0,2022-10-25,0,0.05,40,FUT,NFO-FUT,NFO
2,12581634,49147,FINNIFTY22SEPFUT,FINNIFTY,0,2022-09-27,0,0.05,40,FUT,NFO-FUT,NFO


In [262]:
instruments[f.segment == 'INDICES', [
    "instrument_token", "exchange_token", "tradingsymbol", "name", "instrument_type", "segment", "exchange"]]


,instrument_token,exchange_token,tradingsymbol,name,instrument_type,segment,exchange
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,273929,1070,ALLCAP,BSE INDEX ALLCAP,EQ,INDICES,BSE
1,274185,1071,AUTO,BSE INDEX AUTO,EQ,INDICES,BSE
2,274441,1072,BANKEX,BSE INDEX BANKEX,EQ,INDICES,BSE
3,274697,1073,BASMTR,BSE INDEX BASMTR,EQ,INDICES,BSE
4,274953,1074,BHRT22,BSE INDEX BHRT22,EQ,INDICES,BSE
5,275209,1075,BSE CD,BSE INDEX BSE CD,EQ,INDICES,BSE
6,275465,1076,BSE CG,BSE INDEX BSE CG,EQ,INDICES,BSE
7,275721,1077,BSE HC,BSE INDEX BSE HC,EQ,INDICES,BSE
8,275977,1078,BSE IT,BSE INDEX BSE IT,EQ,INDICES,BSE


# All Futures in NFO

In [263]:
nfo_fut_data = instruments[f.segment=='NFO-FUT',f[:].extend({"underlying": f.name}).remove([f.strike, f.last_price, f.name])]
futures_underlying = nfo_fut_data['underlying'][0, :, dt.by('underlying')].to_list()[0]
print(f"Futures NFO Underlyings: {len(futures_underlying)}")
nfo_fut_data.head(3)


Futures NFO Underlyings: 198


,instrument_token,exchange_token,tradingsymbol,expiry,tick_size,lot_size,instrument_type,segment,exchange,underlying
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,13669122,53395,NIFTY22NOVFUT,2022-11-24,0.05,50,FUT,NFO-FUT,NFO,NIFTY
1,13299458,51951,NIFTY22OCTFUT,2022-10-27,0.05,50,FUT,NFO-FUT,NFO,NIFTY
2,9604354,37517,NIFTY22SEPFUT,2022-09-29,0.05,50,FUT,NFO-FUT,NFO,NIFTY


In [37]:
nse_equity_data = instruments[(f.segment == 'NSE') & (f.exchange == 'NSE') & (f.name != None), [
    "instrument_token", "exchange_token", "tradingsymbol", "name", "instrument_type", "segment", "exchange"]]


In [ ]:

nse_equity_data['futures_underlying'] = f.tradingsymbol

equities = nse_equity_data[0,'tradingsymbol', dt.by('tradingsymbol')].to_list()[0]
print(f"NSE Equities: {len(equities)}")
missing_futures_underlying = set(futures_underlying)-set(equities)
print("Missing Futures", missing_futures_underlying)
nse_equity_data.head(3)

In [23]:
indices_mapping = {
    'NIFTY 50': 'NIFTY',
    'NIFTY BANK': 'BANKNIFTY',
    'NIFTY MID SELECT': 'MIDCPNIFTY',
    'NIFTY FIN SERVICE': 'FINNIFTY',
}

index_include_list = [
    'NIFTY 50', 'NIFTY 500', 'INDIA VIX',
    'NIFTY BANK', 'NIFTY COMMODITIES', 'NIFTY FIN SERVICE', 'NIFTY IT', 'NIFTY INFRA',
    'NIFTY MEDIA', 'NIFTY METAL', 'NIFTY MID SELECT', 'NIFTY OIL AND GAS',
    'NIFTY PHARMA', 'NIFTY PSU BANK', 'NIFTY PSE', 'NIFTY REALTY', 'NIFTY SMLCAP 250', 'NIFTY IND DIGITAL',
    'NIFTY HEALTHCARE', 'NIFTY FMCG'
]

# len(index_include_list), len(set(index_include_list))


In [24]:
nse_index_data = instruments[(f.segment == 'INDICES') & (f.exchange == 'NSE') & (f.name != None), [
        "instrument_token", "exchange_token", "tradingsymbol", "name", "instrument_type", "segment", "exchange"
        ]
    ]

index_inclusions = dt.Frame(tradingsymbol=index_include_list, dummy=[
                            1]*len(index_include_list))
index_inclusions.key = 'tradingsymbol'
nse_index_data = nse_index_data[:, :, dt.join(index_inclusions)][~dt.isna(
    f.dummy), f[:].remove(f.dummy)]

nse_index_data['futures_underlying'] = nse_index_data['tradingsymbol'].to_pandas(
        )['tradingsymbol'].apply(lambda x: indices_mapping.get(x, None))

nse_index_data[~dt.isna(f.futures_underlying),:].head(3)

,instrument_token,exchange_token,tradingsymbol,name,instrument_type,segment,exchange,futures_underlying
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,256265,1001,NIFTY 50,NIFTY 50,EQ,INDICES,NSE,NIFTY
1,260105,1016,NIFTY BANK,NIFTY BANK,EQ,INDICES,NSE,BANKNIFTY
2,257801,1007,NIFTY FIN SERVICE,NIFTY FIN SERVICE,EQ,INDICES,NSE,FINNIFTY


In [8]:
nfo_opt_data = instruments[f.segment == 'NFO-OPT', f[:].extend(
    {"underlying": f.name}).remove([f.last_price, f.name])]
options_underlying = nfo_opt_data['underlying'][0, :, dt.by('underlying')].to_list()[
    0]
print(f"Options NFO Underlyings: {len(options_underlying)}")
nfo_opt_data.head(3)


Options NFO Underlyings: 198


,instrument_token,exchange_token,tradingsymbol,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,underlying
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,12556802,49050,NIFTY22SEP10000CE,2022-09-29,10000,0.05,50,CE,NFO-OPT,NFO,NIFTY
1,12557058,49051,NIFTY22SEP10000PE,2022-09-29,10000,0.05,50,PE,NFO-OPT,NFO,NIFTY
2,14256386,55689,NIFTY22SEP11000CE,2022-09-29,11000,0.05,50,CE,NFO-OPT,NFO,NIFTY


In [9]:
options_include_list = [
    'NIFTY', 
    'BANKNIFTY',
    'ADANIENT',
    'ADANIPORTS',
    #  'AUROPHARMA',
    #  'AXISBANK',
    #  'BAJAJFINSV',
    #  'BAJFINANCE',
    #  'BHARTIARTL',
    #  'DIVISLAB',
    #  'DLF',
    #  'DRREDDY',
    #  'HDFC',
    #  'HDFCBANK',
    #  'ICICIBANK',
    #  'INFY',
    #  'JINDALSTEL',
    #  'JSWSTEEL',
    #  'JUBLFOOD',
    #  'KOTAKBANK',
    #  'L&TFH',
    #  'LALPATHLAB',
    #  'MARUTI',
    #  'RELIANCE',
    #  'SBIN',
    #  'TCS',
 ]

options_inclusions = dt.Frame(underlying=options_include_list, dummy=[
                            1]*len(options_include_list))
options_inclusions.key = 'underlying'
nfo_opt_data = nfo_opt_data[:, :, dt.join(options_inclusions)][~dt.isna(
    f.dummy), f[:].remove(f.dummy)]
nfo_opt_data


,instrument_token,exchange_token,tradingsymbol,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,underlying
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,12556802,49050,NIFTY22SEP10000CE,2022-09-29,10000,0.05,50,CE,NFO-OPT,NFO,NIFTY
1,12557058,49051,NIFTY22SEP10000PE,2022-09-29,10000,0.05,50,PE,NFO-OPT,NFO,NIFTY
2,14256386,55689,NIFTY22SEP11000CE,2022-09-29,11000,0.05,50,CE,NFO-OPT,NFO,NIFTY
3,14256642,55690,NIFTY22SEP11000PE,2022-09-29,11000,0.05,50,PE,NFO-OPT,NFO,NIFTY
4,14999298,58591,NIFTY22SEP12000CE,2022-09-29,12000,0.05,50,CE,NFO-OPT,NFO,NIFTY
5,14999554,58592,NIFTY22SEP12000PE,2022-09-29,12000,0.05,50,PE,NFO-OPT,NFO,NIFTY
6,14999810,58593,NIFTY22SEP13000CE,2022-09-29,13000,0.05,50,CE,NFO-OPT,NFO,NIFTY
7,15000066,58594,NIFTY22SEP13000PE,2022-09-29,13000,0.05,50,PE,NFO-OPT,NFO,NIFTY
8,14041858,54851,NIFTY22SEP14000CE,2022-09-29,14000,0.05,50,CE,NFO-OPT,NFO,NIFTY


In [91]:
from datetime import date, timedelta

today = date.today()
yesterday = date.today() - timedelta(days=1)
START_DATE = date.today() - timedelta(days=365*25)

print(START_DATE, yesterday)
dictionary = {
    'instrument_token': '738561',
    'from_date': date(2000,1,1),
    'to_date': date(2004,12,31),
    'interval': 'day',
    # 'continuous': True,
    'oi': True,
}
kite.historical_data(**dictionary)


1997-10-04 2022-09-27


InputException: interval exceeds limit: 2000 days

In [73]:
kite.ohlc('NSE:RELIANCE')

{'NSE:RELIANCE': {'instrument_token': 738561,
  'last_price': 2332.45,
  'ohlc': {'open': 2360.35, 'high': 2378, 'low': 2327.1, 'close': 2396.25}}}

In [88]:
data = kite.ohlc('NSE:RELIANCE')


In [89]:
df = pd.DataFrame.from_records(data)


In [90]:
df.sort_values('date')

,date,open,high,low,close,volume,oi
0,2000-01-03 00:00:00+05:30,45.72,48.45,45.72,48.45,23150255,0
1,2000-01-04 00:00:00+05:30,49.74,52.33,48.38,52.33,49287678,0
2,2000-01-05 00:00:00+05:30,49.41,55.42,49.41,54.38,139395761,0
3,2000-01-06 00:00:00+05:30,55.63,57.88,55.63,56.66,81466421,0
4,2000-01-07 00:00:00+05:30,56.79,61.20,56.40,60.55,103225855,0
...,...,...,...,...,...,...,...
1252,2004-12-27 00:00:00+05:30,100.10,105.30,97.47,99.51,96810821,0
1253,2004-12-28 00:00:00+05:30,100.09,101.83,99.83,101.55,39878332,0
1254,2004-12-29 00:00:00+05:30,101.87,103.71,101.37,102.31,36002894,0
1255,2004-12-30 00:00:00+05:30,102.80,103.73,100.33,100.70,35049449,0


In [41]:
nse_equity_data[f.tradingsymbol == 'RELIANCE',:]


,instrument_token,exchange_token,tradingsymbol,name,instrument_type,segment,exchange
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,738561,2885,RELIANCE,RELIANCE INDUSTRIES,EQ,NSE,NSE


In [38]:
nse_equity_data['tradingsymbol'].to_list()[0]


['20MICRONS',
 '21STCENMGM',
 '3IINFOLTD',
 '3MINDIA',
 '3PLAND',
 '4THDIM',
 '5PAISA',
 '63MOONS',
 'A2ZINFRA',
 'AAKASH',
 'AAREYDRUGS',
 'AARON-BE',
 'AARTIDRUGS',
 'AARTIIND',
 'AARTISURF',
 'AARVEEDEN',
 'AARVI',
 'AAVAS',
 'ABAN',
 'ABB',
 'ABBOTINDIA',
 'ABCAPITAL',
 'ABFRL',
 'ABMINTLLTD-BE',
 'ABSLAMC',
 'ABSLBAINAV',
 'ABSLBANETF',
 'ABSLNN50ET',
 'ABSLNNINAV',
 'ACC',
 'ACCELYA',
 'ACCURACY',
 'ACE',
 'ACRYSIL',
 'ADANIENT',
 'ADANIGREEN',
 'ADANIPORTS',
 'ADANIPOWER-BE',
 'ADANITRANS',
 'ADFFOODS',
 'ADL-BE',
 'ADORWELD',
 'ADROITINFO',
 'ADSL',
 'ADVANIHOTR',
 'ADVENZYMES',
 'AEGISCHEM',
 'AETHER',
 'AFFLE',
 'AGARIND',
 'AGI',
 'AGRITECH',
 'AGROPHOS',
 'AGSTRA',
 'AHLADA-BE',
 'AHLEAST-BE',
 'AHLUCONT',
 'AIAENG',
 'AIRAN',
 'AIROLAM',
 'AJANTPHARM',
 'AJMERA',
 'AJOONI',
 'AJRINFRA',
 'AKASH',
 'AKG-BE',
 'AKSHAR',
 'AKSHARCHEM',
 'AKSHOPTFBR',
 'AKZOINDIA',
 'ALANKIT',
 'ALBERTDAVD',
 'ALEMBICLTD',
 'ALICON',
 'ALKALI',
 'ALKEM',
 'ALKYLAMINE',
 'ALLCARGO',
 'ALLSEC',


In [3]:
from src.extract_tools.extract_instruments import get_nse_equities, get_nse_options_subset, get_nse_futures

In [5]:
eq = get_nse_equities()

In [60]:
print(eq[:, f.instrument_token].nunique()), eq[:, f.instrument_token].shape


   | instrument_token
   |            int64
-- + ----------------
 0 |             2144
[1 row x 1 column]



(None, (2144, 1))

In [62]:
fut = get_nse_futures()

In [63]:
fut

,instrument_token,exchange_token,tradingsymbol,expiry,tick_size,lot_size,instrument_type,segment,exchange,underlying_tradingsymbol
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,13669122,53395,NIFTY22NOVFUT,2022-11-24,0.05,50,FUT,NFO-FUT,NFO,NIFTY 50
1,13299458,51951,NIFTY22OCTFUT,2022-10-27,0.05,50,FUT,NFO-FUT,NFO,NIFTY 50
2,9604354,37517,NIFTY22SEPFUT,2022-09-29,0.05,50,FUT,NFO-FUT,NFO,NIFTY 50
3,13668866,53394,BANKNIFTY22NOVFUT,2022-11-24,0.05,25,FUT,NFO-FUT,NFO,NIFTY BANK
4,13297154,51942,BANKNIFTY22OCTFUT,2022-10-27,0.05,25,FUT,NFO-FUT,NFO,NIFTY BANK
5,9604098,37516,BANKNIFTY22SEPFUT,2022-09-29,0.05,25,FUT,NFO-FUT,NFO,NIFTY BANK
6,13669378,53396,AARTIIND22NOVFUT,2022-11-24,0.05,850,FUT,NFO-FUT,NFO,AARTIIND
7,13299714,51952,AARTIIND22OCTFUT,2022-10-27,0.05,850,FUT,NFO-FUT,NFO,AARTIIND
8,9607170,37528,AARTIIND22SEPFUT,2022-09-29,0.05,850,FUT,NFO-FUT,NFO,AARTIIND


In [65]:
fut['instrument_token'].nunique()

,instrument_token
,▪▪▪▪▪▪▪▪
0,602


In [66]:
opt = get_nse_options_subset()
print(opt)
opt['instrument_token'].nunique()


     | instrument_token  exchange_token  tradingsymbol         expiry       strike  tick_size  lot_size  instrument_type  segment  exchange  underlying_tradingsymbol
     |            int32  str32           str32                 date32      float64    float64     int32  str32            str32    str32     str32                   
---- + ----------------  --------------  --------------------  ----------  -------  ---------  --------  ---------------  -------  --------  ------------------------
   0 |         12556802  49050           NIFTY22SEP10000CE     2022-09-29    10000       0.05        50  CE               NFO-OPT  NFO       NIFTY 50                
   1 |         12557058  49051           NIFTY22SEP10000PE     2022-09-29    10000       0.05        50  PE               NFO-OPT  NFO       NIFTY 50                
   2 |         14256386  55689           NIFTY22SEP11000CE     2022-09-29    11000       0.05        50  CE               NFO-OPT  NFO       NIFTY 50                
   3

,instrument_token
,▪▪▪▪▪▪▪▪
0,4853


In [74]:
eq

,instrument_token,exchange_token,tradingsymbol,name,instrument_type,segment,exchange
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,264713,1034,HANGSENG BEES-NAV,HANGSENG BEES-NAV,EQ,INDICES,NSE
1,264969,1035,INDIA VIX,INDIA VIX,EQ,INDICES,NSE
2,260617,1018,NIFTY 100,NIFTY 100,EQ,INDICES,NSE
3,264457,1033,NIFTY 200,NIFTY 200,EQ,INDICES,NSE
4,256265,1001,NIFTY 50,NIFTY 50,EQ,INDICES,NSE
5,268041,1047,NIFTY 500,NIFTY 500,EQ,INDICES,NSE
6,265993,1039,NIFTY ALPHA 50,NIFTY ALPHA 50,EQ,INDICES,NSE
7,273673,1069,NIFTY ALPHALOWVOL,NIFTY ALPHALOWVOL,EQ,INDICES,NSE
8,263433,1029,NIFTY AUTO,NIFTY AUTO,EQ,INDICES,NSE


In [2]:
eq = get_nse_equities()[:,f['instrument_token', 'tradingsymbol', 'exchange']]

NameError: name 'get_nse_equities' is not defined

In [84]:


for token in eq['instrument_token'].to_list()[0][0:1]:
    print(token)
    kite.historical_data(token, )


264713


In [47]:
from datetime import date, timedelta
dictionary = {
    'instrument_token': '738561',
    'from_date': date(2001, 1,1),
    'to_date': date(2007,12,31),
    'interval': 'day',
    # 'continuous': True,
    'oi': True,
}

# kite.historical_data(**dictionary)


In [5]:
from time import sleep
from datetime import date, timedelta
from datatable import Frame, dt, f
from src.utils.storage_utils import save, load
from src.extract_tools.extract_instruments import get_nse_equities, get_nse_options_subset, get_nse_futures

eq = get_nse_equities()[:, f['instrument_token', 'tradingsymbol', 'exchange', 'name']]
eq.key = 'instrument_token'
interval = 'day'
for token in eq['instrument_token'].to_list()[0][0:1]:
    chart_data = []
    for year in range(2000,2022,5):
        from_date = date(year, 1, 1)
        to_date = min(date(year+4, 12, 31), date.today() - timedelta(days=1))
        sleep(0.5)
        chart_data.extend(kite.historical_data(token, from_date, to_date, interval))
    data = Frame(chart_data)
    data['instrument_token'] = token
    save(
        data[:,:, dt.join(eq)], 
        eq[f.instrument_token == token, f.tradingsymbol].to_list()[0][0],
        'equities'
    )

2022-09-29 23:14:45,677 - [INFO] - src.utils.storage_utils -     (storage_utils.py).save(28) - equities/21STCENMGM write completed


In [48]:
min(date(2020+4,12,31), date.today() - timedelta(days=1))

datetime.date(2022, 9, 28)

# Get latest equities

In [3]:
from time import sleep
from datetime import date, timedelta
from datatable import Frame, dt, f
from src.utils.storage_utils import save, load
from src.extract_tools.extract_instruments import get_nse_equities, get_nse_options_subset, get_nse_futures

eq = get_nse_equities()[:, f['instrument_token',
                             'tradingsymbol', 'exchange', 'name']]
eq = eq[:, f[:].extend({'exchange_tradingsymbol': f.exchange+':'+f.tradingsymbol})]
list_of_inputs = eq['exchange_tradingsymbol'].to_list()[0]


2022-10-02 14:10:45,201 - [INFO] - src.utils.storage_utils -     (storage_utils.py).load(59) - instruments/instruments20221002 does not exist
2022-10-02 14:10:50,161 - [INFO] - src.utils.storage_utils -     (storage_utils.py).save(28) - instruments/instruments20221002 write completed


In [9]:
quotes_data = {}
for i in range(0, len(list_of_inputs), 400):
    quotes_data.update(kite.quote(*list_of_inputs[i:i+400]))
    sleep(0.5)
quotes_data
# eq.key = 'instrument_token'


{'NSE:20MICRONS': {'instrument_token': 4331777,
  'timestamp': datetime.datetime(2022, 9, 30, 20, 47),
  'last_trade_time': datetime.datetime(2022, 9, 30, 15, 43, 32),
  'last_price': 97.75,
  'last_quantity': 0,
  'buy_quantity': 0,
  'sell_quantity': 0,
  'volume': 0,
  'average_price': 0,
  'oi': 0,
  'oi_day_high': 0,
  'oi_day_low': 0,
  'net_change': 0,
  'lower_circuit_limit': 78.2,
  'upper_circuit_limit': 117.3,
  'ohlc': {'open': 95.5, 'high': 99.3, 'low': 95.5, 'close': 96.35},
  'depth': {'buy': [{'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0}],
   'sell': [{'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0},
    {'price': 0, 'quantity': 0, 'orders': 0}]}},
 'NSE:21STCENMGM':

In [12]:
len(quotes_data.keys())

2176

In [10]:
for ts, quote_data in quotes_data.items():
    quote_data.update(quote_data['ohlc'])
    mapping_quote = {
        'last_price': 'close',
        'last_trade_time': 'date'
    }

    df_eq = Frame({mapping_quote.get(k,k): [v] for k, v in quote_data.items() if k in (
        'open', 'high', 'low', 'last_price', 'volume', 'last_trade_time')})
    df_eq.cbind(eq[f.exchange_tradingsymbol == ts, :])
    past_data = load(
        '21STCENMGM',
        'equities'
    )
    past_data.rbind(df_eq[:, f[:].remove(f.exchange_tradingsymbol)])
    print(df_eq)

    print(past_data.tail(3))
    
    break


   | date                   close  volume     open     high      low  instrument_token  tradingsymbol  exchange  name        exchange_tradingsymbol
   | time64               float64    int8  float64  float64  float64             int32  str32          str32     str32       str32                 
-- + -------------------  -------  ------  -------  -------  -------  ----------------  -------------  --------  ----------  ----------------------
 0 | 2022-09-30T15:43:32    97.75       0     95.5     99.3     95.5           4331777  20MICRONS      NSE       20 MICRONS  NSE:20MICRONS         
[1 row x 11 columns]

   | date                    open     high      low    close  volume  instrument_token  tradingsymbol  exchange  name                     
   | time64               float64  float64  float64  float64   int32             int32  str32          str32     str32                    
-- + -------------------  -------  -------  -------  -------  ------  ----------------  -------------  -----

In [17]:
past_data = load(
    '21STCENMGM',
    'equities'
)
past_data


,date,open,high,low,close,volume,instrument_token,tradingsymbol,exchange,name
,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,2018-01-10T18:30:00,33.4,33.4,33.4,33.4,157,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
1,2018-01-11T18:30:00,34.05,34.05,34.05,34.05,360,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
2,2018-01-14T18:30:00,34.7,34.7,34.7,34.7,795,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
3,2018-01-15T18:30:00,35.35,35.35,35.35,35.35,2735,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
4,2018-01-16T18:30:00,36.05,36.05,35.4,36.05,56033,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
5,2018-01-17T18:30:00,36.75,36.75,36.7,36.75,7957,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
6,2018-01-18T18:30:00,36.9,37.45,36.75,37.45,21575,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
7,2018-01-21T18:30:00,38.15,38.15,37.45,38.15,9893,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE
8,2018-01-22T18:30:00,38.9,38.9,38.2,38.4,5100,1025,21STCENMGM,NSE,21ST CENTURY MGMT SERVICE


In [40]:
interval = 'day'
for token in eq['instrument_token'].to_list()[0][0:1]:
    chart_data = []
    for year in range(2000, 2022, 5):
        from_date = date(year, 1, 1)
        to_date = min(date(year+4, 12, 31), date.today() - timedelta(days=1))
        sleep(0.5)
        chart_data.extend(kite.historical_data(
            token, from_date, to_date, interval))
    data = Frame(chart_data)
    data['instrument_token'] = token
    save(
        data[:, :, dt.join(eq)],
        eq[f.instrument_token == token, f.tradingsymbol].to_list()[0][0],
        'equities'
    )


ValueError: The join frame is not keyed

In [22]:
eq.to_pandas().apply(lambda x:x)

,instrument_token,tradingsymbol,exchange,name
0,264713,HANGSENG BEES-NAV,NSE,HANGSENG BEES-NAV
1,264969,INDIA VIX,NSE,INDIA VIX
2,260617,NIFTY 100,NSE,NIFTY 100
3,264457,NIFTY 200,NSE,NIFTY 200
4,256265,NIFTY 50,NSE,NIFTY 50
...,...,...,...,...
2139,2916865,ZOTA,NSE,ZOTA HEALTH CARE
2140,7436801,ZUARI,NSE,ZUARI AGRO CHEMICALS
2141,979713,ZUARIIND,NSE,ZUARI INDUSTRIES
2142,2029825,ZYDUSLIFE,NSE,ZYDUS LIFESCIENCES


In [25]:
eq[:, f[:].extend({'exchange_tradingsymbol': f.exchange+':'+f.tradingsymbol})]

,instrument_token,tradingsymbol,exchange,name,exchange_tradingsymbol
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,264713,HANGSENG BEES-NAV,NSE,HANGSENG BEES-NAV,NSE:HANGSENG BEES-NAV
1,264969,INDIA VIX,NSE,INDIA VIX,NSE:INDIA VIX
2,260617,NIFTY 100,NSE,NIFTY 100,NSE:NIFTY 100
3,264457,NIFTY 200,NSE,NIFTY 200,NSE:NIFTY 200
4,256265,NIFTY 50,NSE,NIFTY 50,NSE:NIFTY 50
5,268041,NIFTY 500,NSE,NIFTY 500,NSE:NIFTY 500
6,265993,NIFTY ALPHA 50,NSE,NIFTY ALPHA 50,NSE:NIFTY ALPHA 50
7,273673,NIFTY ALPHALOWVOL,NSE,NIFTY ALPHALOWVOL,NSE:NIFTY ALPHALOWVOL
8,263433,NIFTY AUTO,NSE,NIFTY AUTO,NSE:NIFTY AUTO
